# RAG System

In [1]:
import tiktoken
from langchain_openai import OpenAIEmbeddings
import numpy as np
import bs4
from langchain_community.document_loaders import WebBaseLoader

import os
from dotenv import load_dotenv

load_dotenv()

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'

langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
os.environ['LANGCHAIN_API_KEY'] = langchain_api_key

open_api_key = os.getenv("OPEN_API_KEY")

os.environ['OPENAI_API_KEY'] = open_api_key
os.environ['USER_AGENT'] = os.getenv("USER_AGENT")
USER_AGENT = os.getenv("USER_AGENT")

In [2]:
# Documents
question = "What kinds of pets do I like?"
document = "My favorite pet is a cat."

Count tokens

In [3]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string(question, "cl100k_base")

8

In [4]:
from langchain_openai import OpenAIEmbeddings
embd = OpenAIEmbeddings()
query_result = embd.embed_query(question)
document_result = embd.embed_query(document)
len(query_result)


1536

In [5]:
print(query_result)

[-0.0035892894957214594, -0.0030708718113601208, -0.012454708106815815, -0.024972829967737198, -0.028308460488915443, 0.01755327545106411, -0.01554935984313488, -0.008535659871995449, -0.006747357081621885, -0.00487027270719409, -0.003703436581417918, 0.005456861574202776, -0.008351756259799004, -0.001444276887923479, 0.009429811500012875, 0.01955718919634819, 0.03076896257698536, 0.01491521019488573, 0.01570155657827854, -0.015473262406885624, -0.016107412055134773, 3.998118336312473e-05, 0.006664917338639498, -0.024389412254095078, -0.01263861171901226, -0.0032151408959180117, 0.011941046454012394, -0.0013158614747226238, -0.018124010413885117, -0.015815703198313713, 0.051188595592975616, -0.0031025793869048357, -0.022880135104060173, -0.01207421813160181, 0.0022005008067935705, 0.002760138362646103, 0.01438252441585064, 0.004844906739890575, 0.013608860783278942, 0.00035928565193898976, -0.0013491542777046561, 0.020952319726347923, 0.01562545821070671, -0.0045151486992836, 0.0140527

Cosine Similarity

In [6]:
import numpy as np

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

similarity = cosine_similarity(query_result, document_result)
print("Cosine Similarity:", similarity)

Cosine Similarity: 0.8806915835035412


Retrieval (documents in vector space) augmented generation (llm)

In [7]:
#### INDEXING ####

# Load blog
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://mathmod.chnu.edu.ua/",)
    # bs_kwargs=dict(
    #     parse_only=bs4.SoupStrainer(
    #         class_=("post-content", "post-title", "post-header")
    #     )
    # ),
)
blog_docs = loader.load()

In [8]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)
print(splits)

# splits = blog_docs.split(' ')
# print(splits)

[Document(metadata={'source': 'https://mathmod.chnu.edu.ua/', 'title': 'Кафедра математичного моделювання - Кафедра математичного моделювання', 'description': 'Офіційна сторінка кафедри математичного моделювання Чернівецького національного університету імені Юрія Федьковича', 'language': 'uk'}, page_content='Кафедра математичного моделювання - Кафедра математичного моделювання\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nПерейти до основного вмісту\n\n\n\n\n\n\n[email\xa0protected]\n\n\n\r\n                 вул. Університетська 28, каб. 17\r\n            \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nНовини\n\nПро нас'), Document(metadata={'source': 'https://mathmod.chnu.edu.ua/', 'title': 'Кафедра математичного моделювання - Кафедра математичного моделювання', 'description': 'Офіційна сторінка кафедри математичного моделювання Чернівецького національного університету імені Юрія Федьковича', 'language': 'uk'}, pag

In [ ]:
# # Index
# from langchain_openai import OpenAIEmbeddings
# from langchain_community.vectorstores import Chroma
# vectorstore = Chroma.from_documents(documents=splits, 
#                                     embedding=OpenAIEmbeddings())

# retriever = vectorstore.as_retriever()

In [ ]:
# # Index
# from langchain_openai import OpenAIEmbeddings
# from langchain_community.vectorstores import Chroma
# vectorstore = Chroma.from_documents(documents=splits, 
#                                     embedding=OpenAIEmbeddings())


# retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

In [9]:
from langchain.vectorstores import FAISS

vectorstore = FAISS.from_documents(splits, OpenAIEmbeddings())

retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

In [10]:
docs = retriever.get_relevant_documents("Хто такий Ігор Михайлович Черевко?")
print(docs)

C:\Users\vmelnyk2\AppData\Local\Temp\ipykernel_28600\3043429598.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents("Хто такий Ігор Михайлович Черевко?")


[Document(id='d881e50a-786d-48ca-b2a4-4949cb155022', metadata={'source': 'https://mathmod.chnu.edu.ua/', 'title': 'Кафедра математичного моделювання - Кафедра математичного моделювання', 'description': 'Офіційна сторінка кафедри математичного моделювання Чернівецького національного університету імені Юрія Федьковича', 'language': 'uk'}, page_content='Черевко Ігор Михайлович\nЗавідувач кафедри\n\n\n\n\nОстанні новини\n\n\n\n28січ\nВикладачі кафедри взяли участь у конференції зі штучного інтелекту "AI HILLS"\n\n\n\n\n10січ\nМіжнародне стажування викладачів у Сучавському університеті імені Штефана чел Маре')]


In [11]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [12]:
# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [13]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("Хто такий Ігор Михайлович Черевко?")

'Ігор Михайлович Черевко - завідувач кафедри математичного моделювання Чернівецького національного університету імені Юрія Федьковича.'

## RAG system all together

parse all links

In [17]:
#### INDEXING ####

# Load blog
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS

with open('../resources/sets/rag-links.txt', 'r') as file:
    data = file.read().replace('\n', ',')

links = data.split(',')

links_no_pdf = [x for x in links if ".pdf" not in x]

loader = WebBaseLoader(
    web_paths=links_no_pdf[:42]
)
blog_docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)
print(splits)

vectorstore = FAISS.from_documents(splits, OpenAIEmbeddings())

vectorstore.save_local("../models/")

# retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

[Document(metadata={'source': 'https://mathmod.chnu.edu.ua/', 'title': 'Кафедра математичного моделювання - Кафедра математичного моделювання', 'description': 'Офіційна сторінка кафедри математичного моделювання Чернівецького національного університету імені Юрія Федьковича', 'language': 'uk'}, page_content='Кафедра математичного моделювання - Кафедра математичного моделювання\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nПерейти до основного вмісту\n\n\n\n\n\n\n[email\xa0protected]\n\n\n\r\n                 вул. Університетська 28, каб. 17\r\n            \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nНовини\n\nПро нас'), Document(metadata={'source': 'https://mathmod.chnu.edu.ua/', 'title': 'Кафедра математичного моделювання - Кафедра математичного моделювання', 'description': 'Офіційна сторінка кафедри математичного моделювання Чернівецького національного університету імені Юрія Федьковича', 'language': 'uk'}, pag

In [9]:
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import pickle

embd = OpenAIEmbeddings()

with open(f"../models/index.pkl", "rb") as f:
    stored_data = pickle.load(f)

vectorstore = FAISS.load_local(folder_path="../models/", embeddings=embd, allow_dangerous_deserialization=True)

retriever = vectorstore.as_retriever(search_kwargs={"k": 5}) # number of documents retrieved

# Prompt
template = """Answer the question based only on the following context:
{context}

Please, provide as much information as possible!
Do not say words like 'context'
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

# LLM
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [10]:
rag_chain.invoke("Хто такий Ігор Михайлович Черевко?")

'Ігор Михайлович Черевко є завідувачем кафедри математичного моделювання Чернівецького національного університету імені Юрія Федьковича. Він народився у 1956 році і є доктором фізико-математичних наук, професором.'

todo:
- deploy python code with vectorstore
- multiquery 